k8s托管的etcd的灾难恢复

https://etcd.io/docs/v3.4/op-guide/recovery/

```
--name：每个etcd与/etc/kubernetes/manifests/etcd.yaml中的--name=要求一致
--initial-cluster：也就是集群中所有的 initial-advertise-peer-urls 的合集。可以从master3的/etc/kubernetes/manifests/etcd.yaml中获取
--initial-cluster-token：etcd-cluster-1，节点的 token 值，设置该值后集群将生成唯一 id，并为每个节点也生成唯一 id，当使用相同配置文件再启动一个集群时，只要该 token 值不一样，etcd 集群就不会相互影响。
--initial-advertise-peer-urls：建议用于节点之间通信的url，节点间将以该值进行通信。可以从master各自的/etc/kubernetes/manifests/etcd.yaml中获取
```

In [ ]:
# 登录获取最近一次的备份节点，获取备份文件。
kubectl get po -n kube-system -o wide | grep etcd-backup 

# 备份文件上传到所有master

# 所有master执行
systemctl stop kubelet
# 停止etcd，所有master执行
docker ps -a | grep etcd_etcd | awk '{system("docker stop "$1"")}'

# master服务器按顺序执行
ETCDCTL_API=3 etcdctl snapshot restore etcd-backup-2023-05-27.db \
  --name k8s-master1 \
  --initial-cluster k8s-master3=https://192.168.182.17:2380,k8s-master1=https://192.168.182.15:2380,k8s-master2=https://192.168.182.16:2380 \
  --initial-cluster-token etcd-cluster-1 \
  --initial-advertise-peer-urls https://192.168.182.15:2380
mv k8s-master1.etcd/member/* /var/lib/etcd/member/

ETCDCTL_API=3 etcdctl snapshot restore etcd-backup-2023-05-27.db \
  --name k8s-master2 \
  --initial-cluster k8s-master3=https://192.168.182.17:2380,k8s-master1=https://192.168.182.15:2380,k8s-master2=https://192.168.182.16:2380 \
  --initial-cluster-token etcd-cluster-1 \
  --initial-advertise-peer-urls https://192.168.182.16:2380 
mv k8s-master2.etcd/member/* /var/lib/etcd/member/

ETCDCTL_API=3 etcdctl snapshot restore etcd-backup-2023-05-27.db \
  --name k8s-master3 \
  --initial-cluster k8s-master3=https://192.168.182.17:2380,k8s-master1=https://192.168.182.15:2380,k8s-master2=https://192.168.182.16:2380 \
  --initial-cluster-token etcd-cluster-1 \
  --initial-advertise-peer-urls https://192.168.182.17:2380
mv k8s-master3.etcd/member/* /var/lib/etcd/member/

# 启动所有master的kubelet
systemctl start kubelet

# 等待1-2min,检查etcd运行状态，日志，排查错误

# 查看是否有遗留的故障etcd容器，需要删除，所有master执行
docker ps -a | grep etcd_etcd
docker rm 

# 重启其他集群组件，所有master执行
docker ps |grep -E 'k8s_kube-apiserver|k8s_kube-controller-manager|k8s_kube-scheduler' | awk '{system("docker restart "$1"")}'

# 确认所有的集群组件状态正常
kubectl get pods -n kube-system

# 重启所有服务器的kubelet
systemctl restart kubelet